In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("DatasetSplitting").getOrCreate()

# authenticate once
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104281.dfs.core.windows.net",
    "9RU1c3jp0LFpRyz+fXUyQc4pc+RCoF+DolY45sCFCRkPKM4V4ucxzeXZMM247jLYqEkCXCfGxj6k+AStanhkHA=="
)

# path to features_v1 from Lab 3
features_v1_path = (
    "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v1/"
)

df = spark.read.format("delta").load(features_v1_path)
print(f"Total rows: {df.count()}")
df.printSchema()


Total rows: 14970953
root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: string (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- avg_rating_per_book: double (nullable = true)
 |-- num_reviews_per_book: long (nullable = true)



In [0]:
# 70% train, 15% validation, 15% test
train_df, val_df, test_df = df.randomSplit([0.7, 0.15, 0.15], seed=42)

print("Train:", train_df.count())
print("Validation:", val_df.count())
print("Test:", test_df.count())


Train: 10480739
Validation: 2244434
Test: 2245780


In [0]:
base_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/"

train_df.write.format("delta").mode("overwrite").save(base_path + "train/")
val_df.write.format("delta").mode("overwrite").save(base_path + "val/")
test_df.write.format("delta").mode("overwrite").save(base_path + "test/")

print(" Splits saved under gold/features_v2/")


 Splits saved under gold/features_v2/


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("TextFeatureExtraction").getOrCreate()

spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104281.dfs.core.windows.net",
    "9RU1c3jp0LFpRyz+fXUyQc4pc+RCoF+DolY45sCFCRkPKM4V4ucxzeXZMM247jLYqEkCXCfGxj6k+AStanhkHA=="
)

train_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/train/"
train_df = spark.read.format("delta").load(train_path)
train_df.printSchema()


root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- n_votes: integer (nullable = true)
 |-- date_added: string (nullable = true)
 |-- review_length: integer (nullable = true)
 |-- avg_rating_per_book: double (nullable = true)
 |-- num_reviews_per_book: long (nullable = true)



In [0]:
%pip install emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/608.4 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 19.5 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import col, udf, trim, length
from pyspark.sql.types import StringType
import re
import emoji

def clean_text(text):
    if not text:
        return None
    

    text = text.lower()
    
   
    text = re.sub(r"http\S+|www\S+", " <URL> ", text)
    
   
    text = re.sub(r"\d+", " <NUM> ", text)
    

    text = emoji.replace_emoji(text, replace="<EMOJI>")
    
    text = re.sub(r"[^\w\s]", " ", text)
    

    text = re.sub(r"\s+", " ", text).strip()
    

    if len(text) < 10:
        return None
    
    return text

In [0]:
clean_text_udf = udf(clean_text, StringType())

train_cleaned = train_df.withColumn("cleaned_review", trim(clean_text_udf(col("review_text"))))

train_cleaned = train_cleaned.filter(
    col("cleaned_review").isNotNull() & (length(col("cleaned_review")) >= 10)
)

train_cleaned.select("review_text", "cleaned_review").show(10, truncate=False)

print("Text cleaning and normalization (with emojis) completed successfully!")

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
%pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/40.5 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/57.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/1.5 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/803.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.5 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.6 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from pyspark.sql.functions import length, size, split, col
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

train_features = train_cleaned \
    .withColumn("review_length_chars", length(col("cleaned_review"))) \
    .withColumn("review_length_words", size(split(col("cleaned_review"), " ")))

print("Added review_length_chars and review_length_words columns")
train_features.select("cleaned_review", "review_length_words", "review_length_chars").show(5, truncate=False)


Added review_length_chars and review_length_words columns
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:


nltk.download("vader_lexicon")
sid = SentimentIntensityAnalyzer()

sample_pd = train_features.limit(10000).select("review_id", "cleaned_review").toPandas()

sentiment = sample_pd["cleaned_review"].astype(str).apply(sid.polarity_scores)
sentiment_df = pd.DataFrame(list(sentiment))
sentiment_df["review_id"] = sample_pd["review_id"]


sentiment_spark = spark.createDataFrame(sentiment_df)


train_features = train_features.join(sentiment_spark, on="review_id", how="left")

print("Sentiment features added successfully!")



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/spark-278486d6-17e6-4ce9-bbf8-dd/nltk_data...


Sentiment features added successfully!


In [0]:

train_features.select(
    "cleaned_review",
    "review_length_words",
    "review_length_chars",
    "pos",
    "neu",
    "neg",
    "compound"
).show(10, truncate=False)

output_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/text_features/"
train_features.write.format("delta").mode("overwrite").save(output_path)

print(f"Text features (basic + sentiment) saved to: {output_path}")


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd


sample_pd = train_features.limit(10000).select("review_id", "cleaned_review").toPandas()


tfidf = TfidfVectorizer(
    max_features=1000,      
    stop_words="english",    
    ngram_range=(1, 2)       
)

X_tfidf = tfidf.fit_transform(sample_pd["cleaned_review"].astype(str))

tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
tfidf_df["review_id"] = sample_pd["review_id"]
tfidf_df.columns = [re.sub(r"[^A-Za-z0-9_]", "_", c) for c in tfidf_df.columns]

print(f"TF-IDF matrix shape: {tfidf_df.shape}")

tfidf_spark = spark.createDataFrame(tfidf_df)
train_tfidf = train_features.join(tfidf_spark, on="review_id", how="left")

print("Example TF-IDF columns:")
print(tfidf.get_feature_names_out()[:20])  # show first 20 features



TF-IDF matrix shape: (10000, 1001)
Example TF-IDF columns:
['abel' 'ability' 'able' 'absolutely' 'act' 'action' 'actual' 'actually'
 'ada' 'add' 'added' 'admit' 'adult' 'adventure' 'age' 'ago' 'al' 'alert'
 'alive' 'alpha']


In [0]:

tfidf_cols = [c for c in tfidf_spark.columns if c not in ["review_id", "title", "rating", "language"]]

train_tfidf = train_features.alias("a").join(
    tfidf_spark.select(["review_id"] + tfidf_cols).alias("b"),
    on="review_id",
    how="left"
)


dupes = [c for c in train_tfidf.columns if train_tfidf.columns.count(c) > 1]
if dupes:
    print(f"Duplicate columns still exist: {dupes}")
else:
    print("No duplicate or ambiguous columns — join successful!")

print(f"Final column count: {len(train_tfidf.columns)}")

print("Example TF-IDF columns:", tfidf.get_feature_names_out()[:10])


No duplicate or ambiguous columns — join successful!
Final column count: 1018
Example TF-IDF columns: ['abel' 'ability' 'able' 'absolutely' 'act' 'action' 'actual' 'actually'
 'ada' 'add']


In [0]:
output_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/tfidf_features_clean/"
train_tfidf.write.format("delta").mode("overwrite").save(output_path)

print(f"✅ TF-IDF features saved successfully to: {output_path}")


✅ TF-IDF features saved successfully to: abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/tfidf_features_clean/


In [0]:
%pip install -U sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/44.0 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/488.0 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.0/488.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/899.7 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/899.7 MB 57.4 MB/s eta 0:00:16
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/899.7 MB 88.2 MB/s eta 0:00:11
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/899.7 MB 200.9 MB/s eta 0:00:05
   ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/899.7 MB 215.2 MB/s eta 0:00:05
   ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/899.7 MB 231.3 MB/s eta 0:00:04
   ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/899.7 MB 251.3 MB/s eta 0:00:04
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/899.7 MB 256.9 MB/s eta 0:00:04
   ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/899.7 MB 246.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import ArrayType, FloatType
import pandas as pd
from sentence_transformers import SentenceTransformer


spark = SparkSession.builder.appName("GoodreadsSemanticEmbeddings").getOrCreate()

train_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/train/"
train_df = spark.read.format("delta").load(train_path)

train_df.select("review_text").show(5, truncate=100)


+----------------------------------------------------------------------------------------------------+
|                                                                                         review_text|
+----------------------------------------------------------------------------------------------------+
|this takes place after ink exchange also. it focuses on niall's and irial's relationship. niall i...|
|love melissa marrs wicked lovely books and this novella did not let me down! very interesting and...|
|                                            review to come after i get some new year's eve dinner ;)|
|again... this was very entertaining to read. gives you adding insight into the side characters li...|
|                                                              irial and niall... what else is there?|
+----------------------------------------------------------------------------------------------------+
only showing top 5 rows


In [0]:

model = SentenceTransformer("all-MiniLM-L6-v2")


@pandas_udf(ArrayType(FloatType()))
def generate_embeddings_udf(text_series: pd.Series) -> pd.Series:
    text_series = text_series.fillna("").astype(str)
    embeddings = model.encode(text_series.tolist(), show_progress_bar=True)
    return pd.Series([list(e) for e in embeddings])


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [0]:

embedded_df = train_df.withColumn("bert_embedding", generate_embeddings_udf(col("review_text")))


embedded_df.select("review_text", "bert_embedding").show(3, truncate=80)


+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                     review_text|                                                                  bert_embedding|
+--------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|this takes place after ink exchange also. it focuses on niall's and irial's r...|[-0.02169014, -0.032547716, -0.030662846, -0.034730017, 9.279439E-4, 0.063089...|
|love melissa marrs wicked lovely books and this novella did not let me down! ...|[-0.036958616, -0.03652925, 0.01726259, -0.0036482585, -0.050371025, 0.043491...|
|                        review to come after i get some new year's eve dinner ;)|[-0.06400844, 0.037013195, -0.02948186, 0.009618851, 0.019642867, -0.04322739...|
+---------------

In [0]:
row_count = embedded_df.count()
print(f"Total rows in dataset: {row_count}")


Total rows in dataset: 10480831


In [0]:
# Sample 50,000 reviews (as the dataset is way too large)
embedded_sample = embedded_df.limit(50000)

output_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/bert_embeddings_sample/"

embedded_sample.coalesce(4).write.format("delta").mode("overwrite").save(output_path)


In [0]:
from pyspark.sql.functions import col, length, size, split, regexp_replace, when, lower

# Start from your current dataframe
additional_df = df

#  Word & character counts
additional_df = additional_df.withColumn("review_length_words", size(split(col("review_text"), " ")))
additional_df = additional_df.withColumn("review_length_chars", length(col("review_text")))

# Average word length
additional_df = additional_df.withColumn(
    "avg_word_length",
    (col("review_length_chars") / when(col("review_length_words") > 0, col("review_length_words")).otherwise(1))
)

#  Punctuation density (expressiveness indicator)
additional_df = additional_df.withColumn(
    "punctuation_density",
    (length(regexp_replace(col("review_text"), "[^\\p{Punct}]", "")) /
     when(col("review_length_chars") > 0, col("review_length_chars")).otherwise(1))
)

#  Emotional punctuation counts (basic tone markers)
additional_df = additional_df.withColumn("num_questions", length(regexp_replace(col("review_text"), "[^?]", "")))
additional_df = additional_df.withColumn("num_exclaims", length(regexp_replace(col("review_text"), "[^!]", "")))

# Preview the key columns
additional_df.select(
    "review_id",
    "review_length_words",
    "avg_word_length",
    "punctuation_density",
    "num_questions",
    "num_exclaims"
).show(5, truncate=False)

# Save the new version to Gold
output_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/additional_features/"
additional_df.coalesce(4).write.format("delta").mode("overwrite").save(output_path)

print(f"Simplified additional features saved successfully to: {output_path}")


+--------------------------------+-------------------+-----------------+--------------------+-------------+------------+
|review_id                       |review_length_words|avg_word_length  |punctuation_density |num_questions|num_exclaims|
+--------------------------------+-------------------+-----------------+--------------------+-------------+------------+
|a60ae7797bb08be40af2c555b9a7181f|123                |5.666666666666667|0.028694404591104734|0            |0           |
|b2ad6346124613579c779dc0d7c0ac22|97                 |5.422680412371134|0.045627376425855515|0            |0           |
|122f4017fccc67604aa52183a900bede|31                 |4.548387096774194|0.0425531914893617  |1            |1           |
|f17f8ee9c8c959bdb34067a586018bf6|322                |5.419254658385094|0.020630372492836675|0            |1           |
|a0ee890d1e26ceb0ec95a02d34a2efa5|197                |5.345177664974619|0.03798670465337132 |1            |0           |
+-------------------------------

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CombineFeatures").getOrCreate()

base_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/"

# Load datasets
tfidf_df = spark.read.format("delta").load(base_path + "tfidf/")
sentiment_df = spark.read.format("delta").load(base_path + "sentiment/")
additional_df = spark.read.format("delta").load(base_path + "additional_features/")



In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("GoldFeaturesMerge").getOrCreate()

# Re-set your secure access key
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104281.dfs.core.windows.net",
    "9RU1c3jp0LFpRyz+fXUyQc4pc+RCoF+DolY45sCFCRkPKM4V4ucxzeXZMM247jLYqEkCXCfGxj6k+AStanhkHA=="
)


In [0]:
from py4j.protocol import Py4JJavaError

base_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/"
folders = ["text_features", "tfidf_features_clean", "bert_embeddings_sample", "additional_features"]

for folder in folders:
    full_path = base_path + folder + "/"
    print(f"\n🔍 Checking folder: {folder}")
    try:
        df = spark.read.format("delta").load(full_path)
        print(f"✅ Loaded as Delta table ({df.count()} rows, {len(df.columns)} cols)")
    except Py4JJavaError:
        print("⚠️ Not Delta — trying Parquet...")
        try:
            df = spark.read.parquet(full_path)
            print(f"✅ Loaded as Parquet ({df.count()} rows, {len(df.columns)} cols)")
        except Exception as e:
            print(f"❌ Could not load {folder}: {str(e)[:200]}")



🔍 Checking folder: text_features
✅ Loaded as Delta table (10431541 rows, 21 cols)

🔍 Checking folder: tfidf_features_clean
✅ Loaded as Delta table (10431541 rows, 1018 cols)

🔍 Checking folder: bert_embeddings_sample
✅ Loaded as Delta table (50000 rows, 15 cols)

🔍 Checking folder: additional_features
✅ Loaded as Delta table (14970953 rows, 20 cols)


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Reconnect Spark and key if needed
spark = SparkSession.builder.appName("FinalFeatureCombination").getOrCreate()
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104281.dfs.core.windows.net",
    "9RU1c3jp0LFpRyz+fXUyQc4pc+RCoF+DolY45sCFCRkPKM4V4ucxzeXZMM247jLYqEkCXCfGxj6k+AStanhkHA=="
)

# Base path
base_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/"



In [0]:
 #Load each feature dataset
text_df = spark.read.format("delta").load(base_path + "text_features/")
tfidf_df = spark.read.format("delta").load(base_path + "tfidf_features_clean/")
bert_df = spark.read.format("delta").load(base_path + "bert_embeddings_sample/")
additional_df = spark.read.format("delta").load(base_path + "additional_features/")


In [0]:
# Standardize the join key
for df_name, df in zip(["text", "tfidf", "bert", "additional"], [text_df, tfidf_df, bert_df, additional_df]):
    if "review_id" not in df.columns:
        raise ValueError(f"{df_name} is missing 'review_id' column")


In [0]:
# Sequentially merge all datasets on review_id
combined_df = (
    text_df
    .join(tfidf_df, on="review_id", how="left")
    .join(bert_df, on="review_id", how="left")
    .join(additional_df, on="review_id", how="left")
)


In [0]:
final_columns = [
    "review_id", "book_id", "author_id", "user_id", "rating",
    "review_length", "avg_rating_per_book", "num_reviews_per_book",
    "review_length_words", "review_length_chars", "avg_word_length",
    "punctuation_density", "num_questions", "num_exclaims"
] + [c for c in combined_df.columns if c.startswith("tfidf_") or c.startswith("bert_")]

final_df = combined_df.select(*[col(c) for c in final_columns if c in combined_df.columns])


{"ts": "2025-11-10 09:28:16.949", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[AMBIGUOUS_REFERENCE] Reference `book_id` is ambiguous, could be: [`book_id`, `book_id`, `book_id`, `book_id`]. SQLSTATE: 42704\n\nJVM stacktrace:\norg.apache.spark.sql.AnalysisException\n\tat org.apache.spark.sql.errors.QueryCompilationErrors$.ambiguousReferenceError(QueryCompilationErrors.scala:2929)\n\tat org.apache.spark.sql.catalyst.expressions.package$AttributeSeq.resolve(package.scala:372)\n\tat org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveChildren(LogicalPlan.scala:196)\n\tat org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpressionByPlanChildren$1(ColumnResolutionHelper.scala:544)\n\tat org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpression$3(ColumnResolutionHelper.scala:153)\n\tat org.apache.spark.sql.catalyst.analysis.package$.withPosition(package.scala:107)\n\tat org.apache.spark.sql.catalyst.

In [0]:
# Save to Gold layer
output_path = base_path + "final_combined/"
final_df.coalesce(8).write.format("delta").mode("overwrite").save(output_path)

print(f"Final combined feature dataset saved successfully to: {output_path}")
print(f"Total rows: {final_df.count()}, Total columns: {len(final_df.columns)}")

{"ts": "2025-11-10 09:30:27.889", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[AMBIGUOUS_REFERENCE] Reference `book_id` is ambiguous, could be: [`book_id`, `book_id`, `book_id`, `book_id`]. SQLSTATE: 42704\n\nJVM stacktrace:\norg.apache.spark.sql.AnalysisException\n\tat org.apache.spark.sql.errors.QueryCompilationErrors$.ambiguousReferenceError(QueryCompilationErrors.scala:2929)\n\tat org.apache.spark.sql.catalyst.expressions.package$AttributeSeq.resolve(package.scala:372)\n\tat org.apache.spark.sql.catalyst.plans.logical.LogicalPlan.resolveChildren(LogicalPlan.scala:196)\n\tat org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpressionByPlanChildren$1(ColumnResolutionHelper.scala:544)\n\tat org.apache.spark.sql.catalyst.analysis.ColumnResolutionHelper.$anonfun$resolveExpression$3(ColumnResolutionHelper.scala:153)\n\tat org.apache.spark.sql.catalyst.analysis.package$.withPosition(package.scala:107)\n\tat org.apache.spark.sql.catalyst.

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-7499574262201067>, line 3
      1 # Save to Gold layer
      2 output_path = base_path + "final_combined/"
----> 3 final_df.coalesce(8).write.format("delta").mode("overwrite").save(output_path)
      5 print(f"Final combined feature dataset saved successfully to: {output_path}")
      6 print(f"Total rows: {final_df.count()}, Total columns: {len(final_df.columns)}")

File /databricks/spark/python/pyspark/sql/connect/readwriter.py:679, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
    677     self.format(format)
    678 self._write.path = path
--> 679 _, _, ei = self._spark.client.execute_command(
    680     self._write.command(self._spark.client), self._write.observations
    681 )
    682 self._callback(ei)

File /databricks/spark/python/pyspark/sql/connect/client/core.py:1484, in SparkConnect

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark and access key
spark = SparkSession.builder.appName("FinalFeatureCombination").getOrCreate()
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60104281.dfs.core.windows.net",
    "9RU1c3jp0LFpRyz+fXUyQc4pc+RCoF+DolY45sCFCRkPKM4V4ucxzeXZMM247jLYqEkCXCfGxj6k+AStanhkHA=="
)

# Base path
base_path = "abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/"

# Load feature tables
text_df = spark.read.format("delta").load(base_path + "text_features/")
tfidf_df = spark.read.format("delta").load(base_path + "tfidf_features_clean/")
bert_df = spark.read.format("delta").load(base_path + "bert_embeddings_sample/")
additional_df = spark.read.format("delta").load(base_path + "additional_features/")

# Rename duplicate columns (keep only review_id clean)
def rename_columns(df, prefix):
    return df.select(
        *[col(c).alias(f"{prefix}_{c}") if c != "review_id" else col(c) for c in df.columns]
    )

tfidf_df = rename_columns(tfidf_df, "tfidf")
bert_df = rename_columns(bert_df, "bert")
additional_df = rename_columns(additional_df, "add")

# Merge everything safely on review_id
combined_df = (
    text_df
    .join(tfidf_df, on="review_id", how="left")
    .join(bert_df, on="review_id", how="left")
    .join(additional_df, on="review_id", how="left")
)

# Select final metadata and features (no ambiguity)
final_columns = [
    "review_id",
    "book_id",
    "author_id",
    "user_id",
    "rating",
    "review_length",
    "avg_rating_per_book",
    "num_reviews_per_book",
    "add_review_length_words",
    "add_review_length_chars",
    "add_avg_word_length",
    "add_punctuation_density",
    "add_num_questions",
    "add_num_exclaims"
] + [c for c in combined_df.columns if c.startswith("tfidf_") or c.startswith("bert_")]

final_df = combined_df.select(*[col(c) for c in final_columns if c in combined_df.columns])

# Save the final unified dataset
output_path = base_path + "final_combined/"
final_df.coalesce(8).write.format("delta").mode("overwrite").save(output_path)

print(f"✅ Final combined dataset saved to: {output_path}")
print(f"Rows: {final_df.count()}, Columns: {len(final_df.columns)}")


✅ Final combined dataset saved to: abfss://lakehouse@goodreadsreviews60104281.dfs.core.windows.net/gold/features_v2/final_combined/
Rows: 10431541, Columns: 1045
